This generates all the code in the library

In [10]:
HEX = "0123456789ABCDEF" * 16

In [ ]:
# Hex
for i in range(16):
    print(f"/// This denotes the number {i}")
    print(f"pub struct _{HEX[i]};")
    print()

for i in range(16):
    print(f"impl Hex for _{HEX[i]} {{ const NUMBER: u32 = {i}; }}")

In [ ]:
# Hex equal
print("/// This trait is the equal trait on hex numbers. False evaluates to _0 and true evaluates to _1")
print("pub trait HexEqual<H: Hex> { type Output: Binary; }")
for i in range(16):
    for j in range(16):
        if i == j:
            print(f"impl HexEqual<_{HEX[i]}> for _{HEX[j]} {{ type Output = _1; }}")
        else:
            print(f"impl HexEqual<_{HEX[i]}> for _{HEX[j]} {{ type Output = _0; }}")

In [ ]:
# Hex assert eq
print("/// This trait is the equal trait on hex numbers with an assertion quality. False ones are simply not implemented")
print("pub trait HexAssertEqual<H: Hex> { }")
for i in range(16):
    print(f"impl HexAssertEqual<_{HEX[i]}> for _{HEX[i]} {{ }}")

In [ ]:
# Hex add 1
print("/// This is an internal implementation of addition of two number")
print("pub trait HexAdd1<H: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        print(f"impl HexAdd1<_{HEX[i]}> for _{HEX[j]} {{ type Output = _{HEX[i + j]}; type Carry = _{HEX[(i+j)//16]}; }}")

In [ ]:
# Hex add 3
print("/// This is an internal implementation of addition of three number")
print("pub trait HexAdd3<H1: Hex, H2: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        rs = f"_{HEX[i + j]}"
        cr = f"_{HEX[(i+j)//16]}"
        print(f"impl<H: Hex, R: Hex, C0: Hex, Cr: Hex, C_: Hex> HexAdd3<_{HEX[i]}, H> for _{HEX[j]} where H: HexAdd<{rs}, Output = R, Carry = C0>, C0: HexAdd<{cr}, Output = Cr, Carry = C_>, C_: HexAssertEqual<_0> {{ type Output = R; type Carry = Cr; }}")

In [ ]:
# Hex add 4 to 16

for adder in range(5, 17):
    h_where = ", ".join([f"H{i}: Hex" for i in range(1, adder)])
    print(f"/// This is an internal implementation of addition of {adder} number")
    print(f"pub trait HexAdd{adder}<{h_where}> {{ type Output: Hex; type Carry: Hex; }}")
    for i in range(16):
        h_generics = ", ".join([f"H{i}" for i in range(1, adder)])
        h_generics_bound = ", ".join([f"H{i}" for i in range(2, adder-1)])
        result = f"""
impl<{h_generics}, R, C1, C2, Result, Cr, C_> HexAdd{adder}<{h_generics}> for _{HEX[i]} where 
{h_where}, 
R: Hex, 
Result: Hex, 
C1: Hex, 
C2: Hex, 
Cr: Hex, 
C_: Hex, 
H1: HexAdd{adder-2}<{h_generics_bound}, Output = R, Carry = C1>, 
R: HexAdd3<_{HEX[i]}, H{adder-1}, Output = Result, Carry = C2>, 
C1: HexAdd<C2, Output = Cr, Carry = C_>, 
C_: HexAssertEqual<_0> 
{{ type Output = Result; type Carry = Cr; }}"""
        result = result.strip().replace("    ", "").replace("\n", "")
        print(result)
    print()

In [ ]:
# Hex complement
print("/// This represents 15 - x: the hexadecimal complement")
print("pub trait HexNot { type Output: Hex; }")
for i in range(16):
    print(f"""impl HexNot for _{HEX[i]} {{ type Output = _{HEX[15-i]}; }}""")

In [ ]:
# Hex multiply
print("/// This is an internal implementation of multiplication of two hexadecimal")
print("pub trait HexMul<H: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        rs = i * j
        mods = (i * j) // 16
        print(f"impl HexMul<_{HEX[i]}> for _{HEX[j]} {{ type Output = _{HEX[rs]}; type Carry = _{HEX[mods]}; }}")

In [ ]:
# Hex less than
print("/// This is an internal implementation of a < b")
print("pub trait HexLessThan<H: Hex> {type Output: Binary; }")
for i in range(16):
    for j in range(16):
        if i < j:
            print(f"impl HexLessThan<_{HEX[i]}> for _{HEX[j]} {{ type Output = _0; }}")
        elif i == j:
            print(f"impl HexLessThan<_{HEX[i]}> for _{HEX[j]} {{ type Output = _0; }}")
        else:
            print(f"impl HexLessThan<_{HEX[i]}> for _{HEX[j]} {{ type Output = _1; }}")

In [ ]:
# Other hex comparison

print("/// This is an internal implementation of a <= b")
print("pub trait HexLeq<H: Hex> { type Output: Binary; }")
for i in range(16):
    print(f"""impl<H: Hex, A: Binary, B: Binary, C: Binary> HexLeq<H> for _{HEX[i]} where H: HexLessThan<_{HEX[i]}, Output = A>, H: HexEqual<_{HEX[i]}, Output = B>, A: BinOr<B, Output = C> {{ type Output = C; }}""")

print("/// This is an internal implementation of a > b")
print("pub trait HexGreaterThan<H: Hex> { type Output: Binary; }")
for i in range(16):
    print(f"""impl<H: Hex, A: Binary, B: Binary, C: Binary> HexGreaterThan<H> for _{HEX[i]} where H: HexLessThan<_{HEX[i]}, Output = A>, H: HexEqual<_{HEX[i]}, Output = B>, A: BinNor<B, Output = C> {{ type Output = C; }}""")

print("/// This is an internal implementation of a >= b")
print("pub trait HexGeq<H: Hex> { type Output: Binary; }")
for i in range(16):
    print(f"""impl<H: Hex, A: Binary, B: Binary> HexGeq<H> for _{HEX[i]} where H: HexLessThan<_{HEX[i]}, Output = A>, A: BinNot<Output = B> {{ type Output = B; }}""")


In [ ]:
# Test cases for parsing
import random
random.seed(12345)
a = set()
while len(a) < 50:
    r = random.random() * 32
    r = int(2 ** r)
    if r in a:
        continue
    a.add(r)
for i, x in enumerate(sorted(a)):
    print(f"let a{i} = parse_integer!({x}); assert_eq!(a{i}.number(), {x});")

In [ ]:
# Test cases for addition
import random
random.seed(12345)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r + s >= 2 ** 32:
        continue
    a.add((r, s))
for i, (x, y) in enumerate(sorted(a, key=lambda x: x[0] + x[1])):
    print(f"let a{i}_0 = parse_integer!({x}); let a{i}_1 = parse_integer!({y}); let a{i}_2 = parse_integer!({x+y}); add_equal(&a{i}_0, &a{i}_1, &a{i}_2);")

In [ ]:
# Test cases for subtraction
import random
random.seed(23456)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r > s:
        r, s = s, r
    a.add((r, s))
for i, (x, y) in enumerate(sorted(a, key=lambda x: x[1] - x[0])):
    print(f"let a{i}_0 = parse_integer!({x}); let a{i}_1 = parse_integer!({y}); let a{i}_2 = parse_integer!({y - x}); sub_equal(&a{i}_0, &a{i}_1, &a{i}_2);")

In [ ]:
# Test cases for multiplication
import random
random.seed(34567)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r > s:
        r, s = s, r
    a.add((r, s))
for i, x in enumerate(sorted(a, key=lambda x: x[0] * x[1])):
    rs = x[0] * x[1]
    upper = rs // (2 ** 32)
    lower = rs % (2 ** 32)
    result = f"""
        let a{i}_0 = parse_integer!({x[0]});
        let a{i}_1 = parse_integer!({x[1]});
        let a{i}_2 = parse_integer!({upper});
        let a{i}_3 = parse_integer!({lower});
        mul_equal(&a{i}_0,&a{i}_1, &a{i}_2, &a{i}_3);
    """
    print(result.strip().replace("    ", "").replace("\n", " "))

In [92]:
# Test cases for multiplication
import random
random.seed(42069)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r > s:
        r, s = s, r
    if len(a) == 48 and r != s:
        a.add((r, r))
        a.add((s, s))
    else:
        a.add((r, s))
for i, x in enumerate(sorted(a)):
    if x[0] == x[1]:
        result = f"""
            let a{i}_0 = parse_integer!({x[0]});
            let a{i}_1 = parse_integer!({x[1]});
            is_less(&a{i}_0, &a{i}_1, &False);
            is_less(&a{i}_1, &a{i}_0, &False);
        """
    else:
        result = f"""
            let a{i}_0 = parse_integer!({x[0]});
            let a{i}_1 = parse_integer!({x[1]});
            is_less(&a{i}_0, &a{i}_1, &True);
            is_less(&a{i}_1, &a{i}_0, &False);
        """
    print(result.strip().replace("    ", "").replace("\n", " "))

let a0_0 = parse_integer!(1); let a0_1 = parse_integer!(3); is_less(&a0_0, &a0_1, &True); is_less(&a0_1, &a0_0, &False);
let a1_0 = parse_integer!(1); let a1_1 = parse_integer!(1406); is_less(&a1_0, &a1_1, &True); is_less(&a1_1, &a1_0, &False);
let a2_0 = parse_integer!(1); let a2_1 = parse_integer!(211920427); is_less(&a2_0, &a2_1, &True); is_less(&a2_1, &a2_0, &False);
let a3_0 = parse_integer!(3); let a3_1 = parse_integer!(591295195); is_less(&a3_0, &a3_1, &True); is_less(&a3_1, &a3_0, &False);
let a4_0 = parse_integer!(4); let a4_1 = parse_integer!(98); is_less(&a4_0, &a4_1, &True); is_less(&a4_1, &a4_0, &False);
let a5_0 = parse_integer!(4); let a5_1 = parse_integer!(15483789); is_less(&a5_0, &a5_1, &True); is_less(&a5_1, &a5_0, &False);
let a6_0 = parse_integer!(5); let a6_1 = parse_integer!(31); is_less(&a6_0, &a6_1, &True); is_less(&a6_1, &a6_0, &False);
let a7_0 = parse_integer!(7); let a7_1 = parse_integer!(4675); is_less(&a7_0, &a7_1, &True); is_less(&a7_1, &a7_0, &False);
l